# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Importing Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II.  Apache Cassandra coding portion of your project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Let's create queries to ask the following three questions of the data

#### Before we start let's recap
A primary key in Cassandra consists of one or more partition keys and zero or more clustering key components. Partition key is responsible for data distribution across the nodes; it is a minimum-specifier to query data using 'WHERE' clause. The Clustering Key is responsible for data sorting within the partition. Finally, the Composite/Compound Key is just any multiple-column key.

We always place the partition key first and then the clustering key. 

(Sources: https://www.baeldung.com/cassandra-keys, https://stackoverflow.com/questions/24949676/difference-between-partition-key-composite-key-and-clustering-key-in-cassandra) 

### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Since we want to find the data based on sessionId and itemInSession, we need to put those two columns in Primary key as cassandra uses the consistent hashing technique in the where clause with a partition key to identify the exact partition range and node within the cluster (following: https://www.scylladb.com/glossary/cassandra-partition-key/).

In [8]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS songs
                    (sessionId INT, 
                    itemInSession INT,
                    artist TEXT, 
                    song_title TEXT, 
                    song_duration DOUBLE,
                    PRIMARY KEY(itemInSession, sessionId))""")
except Exception as e:
    print(e)

In [9]:
file = "event_datafile_new.csv"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO songs (sessionId, itemInSession, artist, song_title, song_duration) "
        query = query + """VALUES (%s, %s, %s, %s, %s)"""
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Let's verify that the data has been inserted into the table

In [10]:
query = "SELECT artist, song_title, song_duration  FROM songs WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.song_duration)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Here our Primary key will consist of Partition keys (to allow to use the WHERE clause) and a Clustering Key to perform sorting. Hence, our composite key (a multiple column key) will consist of userid, sessionid and itemInSession.

In [11]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS users
                    (userId INT,
                    sessionId INT,
                    itemInSession INT,
                    artist TEXT,
                    song_title TEXT,
                    user_firstname TEXT, 
                    user_lastname TEXT,
                    PRIMARY KEY((userId, sessionId), itemInSession))""")
except Exception as e:
    print(e)   

In [12]:
file = "event_datafile_new.csv"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO users (userId, sessionId, itemInSession, artist, song_title, user_firstname, user_lastname) "
        query = query + """VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        session.execute(query, (int(line[-1]), int(line[-3]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Let's verify that the data has been inserted into the table

In [13]:
query = "SELECT  artist, song_title, user_firstname, user_lastname FROM users WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.user_firstname, row.user_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Finally, here the Primary key will be song_title and userId. UserID is used in order to select unique ids of users and hence unique users.

In [14]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS listeners
                    (song_title TEXT,
                    userId INT,
                    user_firstname TEXT, 
                    user_lastname TEXT,  
                    PRIMARY KEY(song_title, userId))""")
except Exception as e:
    print(e)    

In [15]:
file = "event_datafile_new.csv"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO listeners (song_title, userId, user_firstname, user_lastname) "
        query = query + """VALUES (%s, %s, %s, %s)"""
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Let's verify that the data has been inserted into the table

In [16]:
query = "SELECT user_firstname, user_lastname FROM listeners WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user_firstname, row.user_lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Droping the tables before closing out the sessions

In [17]:
try:
    session.execute("DROP TABLE IF EXISTS songs")
    
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE IF EXISTS users")
    
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS listeners")
    
except Exception as e:
    print(e)

### Closing the session and clustering connection¶

In [18]:
session.shutdown()
cluster.shutdown()